# Librerias requeridas

In [ ]:
# Generales
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from datetime import *
import time
from scipy import stats
from statistics import mode

#Graficos
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
%matplotlib inline

#Models y preprocesamiento
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, train_test_split, KFold, ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb

# Metricas
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve

#series temporales
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.model_selection import TimeSeriesSplit 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

# Otros
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import make_pipeline

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install xgboost

# Leactura de los set de datos

In [ ]:
## para leer en kaggle
df_index = pd.read_csv('/kaggle/input/evol-metricas-prestadores/index_50.csv')
df_metricas = pd.read_csv('/kaggle/input/evol-metricas-prestadores/metricas_prestador_plan.csv')
df_procedimientos = pd.read_csv('/kaggle/input/evol-metricas-prestadores/procedimientos.csv')

## para leer en notebook local
# df_index = pd.read_csv('data/index_50.csv')
# df_metricas = pd.read_csv('data/metricas_prestador_plan.csv')
# df_procedimientos  = pd.read_csv('data/procedimientos.csv')



print(df_metricas.shape)

print(len(df_metricas.Prestador_ID.unique()))

df_metricas.sample(4)

# (Prestador_ID	Periodo 	Agrupador_Plan) unico!

In [ ]:
print(sum(df_metricas.Periodo == '2020-12-01'))
df_metricas.columns

# EDA

In [ ]:
sns.histplot(df_metricas[df_metricas.Periodo=='2018-11-01'].afiliado_count, log_scale=True);
# sns.boxplot(df_metricas[df_metricas.Periodo=='2018-11-01'].afiliado_count);


In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
sns.set_theme(style="whitegrid")
sns.barplot(y = 'afiliado_nunique', x = 'Periodo', data = df_metricas, palette="Blues_d", ax = ax).\
    set(title='Evolución mensual - Todos los prestadores')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()
fig, ax = plt.subplots(figsize=(15,5))

sns.barplot(y = 'afiliado_prestaciones_afil_unic', x = 'Periodo', data = df_metricas, palette="Blues_d", ax = ax).\
    set(title='Evolución mensual - Todos los prestadores')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

El gráfico anterior permite visualizar claramente el impacto de la pandemia y la cuarentena implementada a mediados de marzo 2020

## Oferta de servicios de los prestadores según PLAN
NO TODOS LOS PRESTADORES OFRECEN SERVICIOS LOS 30 MESES NI A TODOS LOS PLANES

In [ ]:
len(df_metricas.Periodo.unique())

In [ ]:
aa = df_metricas.groupby(['Prestador_ID', 'Agrupador_Plan']).Periodo.count()
sns.histplot(aa.values);

In [ ]:
print(len(df_metricas.Agrupador_Plan.unique()))
aa = df_metricas.groupby(['Prestador_ID']).Periodo.count()
sns.histplot(aa.values);

In [ ]:
df_metricas.groupby('Prestador_ID').Periodo.nunique().reset_index().head(20)

In [ ]:
13*30 ## cantidad de prestadores x cantidad de meses

No todos los prestadores atienden a todos los planes. 
Abajo una lista con la cantidad de prestadores por cada grupo de plan:

In [ ]:
df = df_metricas.copy()
df.drop(columns = ['Prestador_ID'], inplace = True)
col_numericas = [i for i in df.columns if df.loc[:,i].apply(np.isreal).all()]
col_numericas

In [ ]:
# ### Creamos la matriz de correlacion y ordenos de mayor a menor
ccc = df[col_numericas].corr()
# ccc

plt.figure(figsize=(20, 8))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(ccc, dtype=np.bool))
heatmap = sns.heatmap(ccc, mask=mask, vmin=-0.5, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

In [ ]:
ll = ['afiliado_nunique',
 'afiliado_count',
 'sexo_afiliado_prop_mujeres',
 'monto_prestacion_q0.50',
#  'monto_prestacion_q0.95',
#  'edad_afil_median',
 'edad_afil_q0.50',
#  'edad_afil_q0.95',
#  'permanencia_median',
 'permanencia_q0.50',
# 'Agrupador_Plan', 
#  'permanencia_q0.95'
     ]
df = df_metricas[ll].sample(100)
plt.figure(figsize=(20, 10))
pplot = sns.pairplot(df, kind="hist", corner=True);
# pplot = sns.pairplot(df);

## Inflación efectiva de la OS

In [ ]:
plt.figure(figsize=(10,5))
g = sns.lineplot(
    data=df_index,
    x='Periodo',
    y= 'Index_50',
    palette='Set1'
)
g.set_xticklabels(labels=df_index.Periodo, rotation=90)
plt.show()

## Influencia de la EDAD en el CONSUMO

In [ ]:
## Cual es la edad de cada plan
aa = df_metricas.groupby('Agrupador_Plan').agg({'edad_afil_q0.50' : 'mean'}).reset_index(drop = False)
aa['anios'] = round(aa['edad_afil_q0.50']/12,1)
aa.sort_values(by = 'anios', ascending = False)
# df_metricas['edad_afil_q0.50']

In [ ]:
## Cuales son los prestadores q atienden a gente mayor

aa = df_metricas.groupby('Prestador_ID').agg({'edad_afil_q0.50' : 'mean'}).reset_index(drop = False)
aa['anios'] = round(aa['edad_afil_q0.50']/12,1)
aa.sort_values(by = 'anios', ascending = False)



# Feature Engeneering

In [ ]:
## de los montos factoramos nos quedamos solo con la MEDIANA que usaremos con target

df_metricas.drop(['monto_prestacion_median', 'monto_prestacion_q0.25', 'monto_prestacion_q0.75', 'monto_prestacion_q0.95'], axis = 1,inplace = True)


In [ ]:
df_metricas.info()

### Nuevas variables:
Contruimos nuevas variables relacionadas con el costo: 
+ Costo por afiliado
+ Costo por afiliado actualizado por indice de imflación
+ Log (base 10) del costo

In [ ]:
df_metricas['costo_afil_uniq'] = df_metricas['monto_prestacion_q0.50'] / df_metricas['afiliado_nunique']
# sns.histplot(df_metricas['costo_afil_unico']);

In [ ]:
df_last = df_metricas.merge(df_index[['Periodo', 'Index_50']], on = 'Periodo', how = 'left')
df_last['costo_afil_uniq_A'] = round(df_last['costo_afil_uniq'] * df_last['Index_50'],2)
df_last.sample(3)

In [ ]:
df_last['log_costo_afil_uniq'] = df_last['costo_afil_uniq'].apply(lambda x : x if x == 0 else np.log10(x))
df_last['log_costo_afil_uniq_A'] = df_last['costo_afil_uniq_A'].apply(lambda x : x if x == 0 else np.log10(x))
sns.histplot(df_last['log_costo_afil_uniq'], bins = 30);
sns.histplot(df_last['log_costo_afil_uniq_A'], bins = 30, color = 'red', kde = True);

In [ ]:
df_last_dummies = pd.get_dummies(df_last, columns = ['Prestador_ID', 'Agrupador_Plan'], prefix = ['prestador', 'plan']).drop(['monto_prestacion_q0.50', 'Periodo'], axis = 1)

In [ ]:
# for periodo in df_metricas.Periodo.unique():
#     mask_mes = df_metricas.Periodo == periodo
#     df_metricas[mask_mes]
y = df_last['monto_prestacion_q0.50']
X = df_last_dummies
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# MODELOS

En los modelos que se presentan se busca predecir la cantidad facturada por un prestador a partir del uso de la información disponible del resto de los prestadores. 

El objetivo de estos modelos es, en todos los casos, detectar anomalías en la liquidación de un prestador (desviaciones respecto a la predcicción superiores a un umbral).

Se trata de un anáisis retrospectivo

In [ ]:
## REGRESION LINEAL como modelo base

In [ ]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_train = model_lr.predict(X_train)
y_pred_test = model_lr.predict(X_test)

In [ ]:
xx = np.linspace(0, max(y_train), 5)
sns.lineplot(x = xx, y = xx, color = 'black')


sns.scatterplot(x  = y_train, y = y_pred_train, color = 'blue');
sns.scatterplot(x  = y_test, y = y_pred_test, color = 'red');



## Modelos basados en el uso adicional del detalle de los Procedimientos 

In [ ]:
print(df_procedimientos.shape)
df_procedimientos.groupby(['Prestador_ID', 'grupo_procedimiento']).count().head(15)

Se hace un join entre el data set de metricas y el de procedimientos

In [ ]:
df_metricas.shape

In [ ]:
df_metr_proc = pd.merge(df_metricas,df_procedimientos, how = 'inner', on = ['Prestador_ID','Periodo','Agrupador_Plan'])
df_metr_proc.sample(4)
df_m_p_group = df_metr_proc.groupby(['Prestador_ID','Periodo','grupo_procedimiento']).agg({'cant_gp' : ['sum','mean']}).reset_index()
df_m_p_group.columns = ['Prestador_ID','Periodo','grupo_procedimiento','suma_proc','media_proc']
df_m_p_group

In [ ]:
df_metr_proc = pd.merge(df_metricas,df_procedimientos, how = 'inner', on = ['Prestador_ID','Periodo','Agrupador_Plan'])
df_metr_proc.sample(4)
df_m_p_group = df_metr_proc.groupby(['Prestador_ID','Periodo','grupo_procedimiento']).mean()

#df_m_p_group.columns = ['Prestador_ID','Periodo','grupo_procedimiento','suma_proc','media_proc']
df_m_p_group = df_m_p_group.reset_index()

df_m_p_group

In [ ]:
df_last.shape

In [ ]:
df_last.notna().count()

In [ ]:
df_m = df_last.copy()
df_m.drop(columns = ['Prestador_ID','Periodo'], inplace = True)
col_numericas_m = [i for i in df_last.columns if df_last.loc[:,i].apply(np.isreal).all()]
col_numericas_m

In [ ]:
# ### Creamos la matriz de correlacion y ordenos de mayor a menor
ccc_m = df_last[col_numericas_m].corr()
# ccc

plt.figure(figsize=(20, 8))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(ccc_m, dtype=np.bool))
heatmap = sns.heatmap(ccc_m, mask=mask, vmin=-0.5, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Matriz correlación Métricas y Procedimientos', fontdict={'fontsize':18}, pad=16);

## Comparación de multiples regresores

- Elimino el ruido de la muestra donde solo me interesan los valores mayores a 1
- Esto lo hacemos quitando los registros con log de costo por afiliado < 1, es decir menos de $10 por afiliado (son prestadores de bajo costo, osea no nos interesan)


In [ ]:
df_reg=df_last[df_last['log_costo_afil_uniq_A']>1]

df_reg_dummies = pd.get_dummies(df_reg, columns = ['Prestador_ID', 'Agrupador_Plan'], prefix = ['prestador', 'plan']).drop(['monto_prestacion_q0.50', 'Periodo'], axis = 1)

Como variable target elegimos el log del costo por afiliado y de las variables dependientes eliminamos el resto de las métricas relacionas explicitamente con el consumo

In [ ]:
y = df_reg['log_costo_afil_uniq_A']
X = df_reg_dummies.drop(columns=['costo_afil_uniq','costo_afil_uniq_A','log_costo_afil_uniq','log_costo_afil_uniq_A'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print("X_train Shape: {}".format(X_train.shape))
print("X_test Shape: {}".format(X_test.shape))

Defininos una función de validacion donde divido la muestra en 5 con shuffle y a la media del error cuadrado como variable scrore

In [ ]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

Definimos los modelos que utilizaremos

In [ ]:
regresores= {}

-  **LASSO  Regression**  : 

In [ ]:
regresores.update({'lasso' : make_pipeline(StandardScaler(), Lasso(alpha =0.5, random_state=1))})

- **Elastic Net Regression** :

In [ ]:
regresores.update({'e_net' : make_pipeline(StandardScaler(), ElasticNet(alpha=0.5, l1_ratio=.9, random_state=3))})

- **Ridge Regression** :

In [ ]:
regresores.update({'ridge' : make_pipeline(StandardScaler(), linear_model.Ridge(alpha=0.5))})

- **SVR Regression** :

In [ ]:
regresores.update({'svr' : make_pipeline(StandardScaler(), SVR(kernel='poly'))})

- **Xgboost Regression** :

In [ ]:
regresores.update({'xgb_r' : xgb.XGBRegressor(objective ='reg:squarederror',n_estimators = 10, seed = 123)})

- **Gradient Boosting Regression** :

In [ ]:
regresores.update({'gbr' : GradientBoostingRegressor()})

- **Random Forest Regression** :

In [ ]:
regresores.update({'r_f' : RandomForestRegressor()})

Guardamos todas las metricas en un data frame para comparar las performances

In [ ]:
perf=pd.DataFrame()

In [ ]:
a=0
for i in regresores.keys():
    score = rmsle_cv(regresores[i])
    perf.loc[a,'regresor'] = i
    print(str(i)+"\n score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
    perf.loc[a,'media']= score.mean()
    perf.loc[a,'desvio']= score.std()
    a+=1

Definimos que el mejor regresor será aquel que obtenga el score y desvío menor

In [ ]:
perf.media.min()

In [ ]:
mejor_reg=perf[perf.desvio==perf.desvio.min()].reset_index().loc[0,'regresor']
print(regresores[mejor_reg])
regresores[mejor_reg].fit(X_test,y_test)

xx = np.linspace(0, max(y_train), 5)
sns.lineplot(x = xx, y = xx, color = 'black')
sns.scatterplot(x  = y_test, y = regresores[mejor_reg].predict(X_test), color = 'blue');

Mediante un histogramos buscaremos donde se forma el "codo" que se aleja de la media de los valores de la predicción

In [ ]:
desvio_rel=((-regresores['gbr'].predict(X_test)+y_test)/y_test)*100

In [ ]:
sns.histplot(desvio_rel,  color = 'purple');

In [ ]:
sns.histplot(desvio_rel,  color = 'purple', cumulative = True, stat = 'density');

# FORECAST DEL CONSUMO POR AFILIADO: Series de tiempo

In [ ]:
# Función que plotea la serie:
def plot_ts(df, x, y, title="", xlabel='Fecha', ylabel='Valor', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.xticks(rotation=45 )
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

# df_last.head(3)

## Valor por prestador y/o global y/u otro

En la siguiente celda definimos el dataframe 'ts' según la variable que querramos predecir

In [ ]:
## Listado de 10 prestadores más 'presentes' (meses x planes)
df = df_last.copy()
df['time'] = pd.to_datetime(df['Periodo'])
# df.set_index(keys = 'time', inplace = True)
df.Prestador_ID.value_counts().head(10)

In [ ]:
mask_prestador = df.Prestador_ID == 158
# mask_prestador = df.Prestador_ID == df.Prestador_ID  ## TODOS LOS PRESTADORES
df=df[mask_prestador]
# mask_prestador.sum()

ts = df[['time','log_costo_afil_uniq_A']].groupby('time').mean().round(2)
ts.columns = ['valor']
ts = ts.iloc[:-1,:]
plot_ts(ts, x=ts.index, y=ts.valor,\
        title='media del log(costo por afiliado) ')

In [ ]:
ts['year'] = ts.index.year
# ts['month'] = ts.index.month
ts['month'] = [d.strftime('%b') for d in ts.index]
ts["timeIndex"] = pd.Series(np.arange(len(ts['valor'])), index=ts.index)
ts["timeIndex_sq"] = ts["timeIndex"]**2
# ts.head(3)
# Creamos variables dummy para los meses:
dummies_mes = pd.get_dummies(ts['month'], drop_first=True)

# Hacemos el join entre el DataFrame con la serie de tiempo y las dummies:
ts = ts.iloc[:, :5]
ts = ts.join(dummies_mes)
ts.tail(4)

In [ ]:
ts_train, ts_test = train_test_split(ts, test_size=3, random_state=1, shuffle=False)
# ts_train.tail()

### lineal

In [ ]:
model_linear = smf.ols('valor ~ timeIndex', data = ts_train).fit()
# model_linear.summary()

## para train
ts_train["LinearTrend"] = model_linear.predict(ts_train.timeIndex)
ts_train.plot(kind = "line", y = ["valor","LinearTrend"]);
# para test
ts_test["LinearTrend"] = model_linear.predict(ts_test.timeIndex)
ts_test.plot(kind = "line", y = ["valor","LinearTrend"]);

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

# Guardamos el resultado en un DataFrame:
df_Results = pd.DataFrame(columns = ["Model", "RMSE"])
df_Results.loc[0, "Model"] = "LinearTrend"
df_Results.loc[0, "RMSE"] = RMSE(ts_test.LinearTrend, ts_test.valor)
df_Results

### cuadratico

In [ ]:
model_quadratic = smf.ols('valor ~ timeIndex + timeIndex_sq', data = ts_train).fit()
# model_quadratic.summary()

## R2 es similar al modelo lineal
## la componente cuadratica NO RESULTA SIGNIFICATIVA (ver p-valor)
## para train
ts_train["QuadraticTrend"] = model_quadratic.predict(ts_train[['timeIndex','timeIndex_sq']])
ts_train.plot(kind = "line", y = ["valor","QuadraticTrend"]);
## para test
ts_test["QuadraticTrend"] = model_quadratic.predict(ts_test[['timeIndex','timeIndex_sq']])
ts_test.plot(kind = "line", y = ["valor","QuadraticTrend"]);

In [ ]:
# Guardamos el resultado en un DataFrame:
df_Results.loc[1, "Model"] = "QuadraticTrend"
df_Results.loc[1, "RMSE"] = RMSE(ts_test.QuadraticTrend, ts_test.valor)
df_Results

### modelo lineal con estacionalidad

In [ ]:
model_lin_est = smf.ols('valor ~ timeIndex + Aug + Dec + Feb + Jan + Jul + Jun + Mar + May + Nov + Oct + Sep',\
                          data = ts_train).fit()
model_lin_est.summary()

In [ ]:
ts_train['model_lin_est'] = model_lin_est.predict(ts_train[["timeIndex", \
                                              "Aug", "Dec", "Feb", "Jan",\
                                               "Jul", "Jun", "Mar", "May",\
                                                "Nov", "Oct", "Sep"]])


ts_test['model_lin_est'] = model_lin_est.predict(ts_test[["timeIndex", \
                                              "Aug", "Dec", "Feb", "Jan",\
                                               "Jul", "Jun", "Mar", "May",\
                                                "Nov", "Oct", "Sep"]])
ts_train.plot(kind = "line", y = ['valor', 'model_lin_est']);
ts_test.plot(kind = "line", y = ['valor', 'model_lin_est']);

In [ ]:
# Guardamos el resultado en un DataFrame:
df_Results.loc[2, "Model"] = "model_lin_est"
df_Results.loc[2, "RMSE"] = RMSE(ts_test.model_lin_est, ts_test.valor)
df_Results

### smoothing con cross validation

In [ ]:
model_exp_smoothing = SimpleExpSmoothing(ts_train.valor).fit(smoothing_level=1., optimized=False)
ts_train.plot(kind = "line", y = "valor")
model_exp_smoothing.fittedvalues.plot();

In [ ]:
# tscv = TimeSeriesSplit(n_splits=5) ## instaciamos un objeto TimeSeriesSplit

def timeseriesCVscore_exp_smoot(alpha, series):
    """
        Returns error on CV  
        
        params - vector of parameters for optimization
        series - dataset with timeseries
        slen - season length for Holt-Winters model
    """
    # creamos un array de errores:
    errors = []
    
    values = series.valor
    
    # instanciamos el objeto que realiza el tscv:
    tscv = TimeSeriesSplit(n_splits=5) 
    
    # Aplicamos cross validation:

    for train, test in tscv.split(values):
    
        model = SimpleExpSmoothing(values[train]).fit(smoothing_level=alpha,\
                                                             optimized=False)
        
        predictions = model.forecast(len(test))
        actual = values[test]
    
        error = mean_squared_error(predictions, actual)
        errors.append(error)
        
    return np.mean(np.array(errors))

In [ ]:
alphas = [0.3, 0.4, 0.5, 0.6, 0.9,  1.03, 1.1, 1.125, 1.15, 1,7, 1.175, 1.8, 1.2]
errors = []

for alpha in alphas:
    error = timeseriesCVscore_exp_smoot(alpha, ts_train)
    errors.append(error)

print('Alpha óptimo:', alphas[np.argmin(errors)]) 
print('\nEste valor depende de el o los prestador seleccionados para hacer el forecast')

In [ ]:
# Entrenamos el modelo optimizado:
# alphas[np.argmin(errors)]
model_exp_smoothing = SimpleExpSmoothing(ts_train.valor, initialization_method = 'heuristic').fit(smoothing_level= 0, optimized=False)
ts_test["Simple_Smoothing"] = model_exp_smoothing.forecast(4)
# ts_test.tail(3)
res_model = ts_train['valor'] - ts_train['QuadraticTrend']
plt.plot(ts_train.timeIndex, res_model, '-');

In [ ]:
# Testeamos la estacionariedad de los residuos:
result = adfuller(res_model)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in  result[4].items():
    print('Valor crítico %s: %.2f' % (key,value))
## En general para ninguno de los modelos puedo rechazar la hipotesis nulo 
## (no podemos garantizar que la serie de los residuos sea estacionaria)

In [ ]:
## calculamos la autocorrelacion del residuo (lag correlation)
lag_acf = acf(res_model, nlags = 5)
ACF = pd.Series(lag_acf)
ACF.plot(kind = "bar");

In [ ]:
lag_pacf = pacf(res_model, nlags=6);
PACF = pd.Series(lag_pacf)
PACF.plot(kind = "bar");

In [ ]:
# Corremos la función con nuestra serie res_log:
def tsplot(y, lags=None, figsize=(12, 7), style='bmh'):
    """ 
        Plotea la serie de tiempo, el ACF y PACF y el test de Dickey–Fuller
        
        y - serie de tiempo
        lags - cuántos lags incluir para el cálculo de la ACF y PACF
        
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        
        # definimos ejes
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        
        # obtengo el p-value con h0: raiz unitaria presente
        p_value = sm.tsa.stattools.adfuller(y)[1]
        
        ts_ax.set_title('Análisis de la Serie de Tiempo\n Dickey-Fuller: p={0:.5f}'\
                        .format(p_value))
        
        # plot de autocorrelacion
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        # plot de autocorrelacion parcial
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()
tsplot(res_model, lags=7)

In [ ]:
# Instanciamos el modelo ARIMA ( en este caso d = 1 ya que no pudimos validar que el residuo sea estacionario)
model_ARIMA = ARIMA(res_model, order=(3,1,2)) ##The (p,d,q) order of the model for the number of AR parameters, differences, and MA parameters to use.

# Estimo el modelo:
results_ARIMA = model_ARIMA.fit()
# print(results_ARIMA.summary())
# results_ARIMA.fittedvalues.head()
res_model.plot()
results_ARIMA.fittedvalues.plot();
results_ARIMA.plot_predict(end=len(ts['valor']));

In [ ]:
## analizo los residuos del modelo ARIMA
res_ARIMA =  results_ARIMA.fittedvalues - res_model
res_ARIMA.plot();

In [ ]:
predictions_ARIMA, se, conf = results_ARIMA.forecast(len(ts_test['valor']), alpha=0.05)
ts_train.head()

In [ ]:
ts_train['lin_model_ARIMA'] = ts_train['LinearTrend'] + results_ARIMA.fittedvalues
ts_test['lin_model_ARIMA'] = ts_test['LinearTrend'] + predictions_ARIMA
ts_train.plot(kind = "line", y = ['valor', 'lin_model_ARIMA']);
ts_test.plot(kind = "line", y = ['valor', 'lin_model_ARIMA']);

In [ ]:
# Guardamos el resultado en un DataFrame:
df_Results.loc[3, "Model"] = "con ARIMA"
df_Results.loc[3, "RMSE"] = RMSE(ts_test.lin_model_ARIMA, ts_test.valor)
df_Results